In [3]:
import matplotlib.pyplot as plt
import numpy as np
from fractions import Fraction
from sympy import Point, Line
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

In [13]:
def primal_line_to_dual_point(p1, p2):
    denominator = p1[0] * p2[1] - p1[1] * p2[0]
    if denominator == 0:
        raise ValueError("Points are linearly dependent or degenerate (denominator = 0).")
    return (
        (p1[1] - p2[1]) / denominator, 
        (p2[0] - p1[0]) / denominator
    )

def orientation(p1, p2, p3):
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    
    # Using Fraction for exact rational arithmetic
    val = (Fraction(x2 - x1) * Fraction(y3 - y1)) - (Fraction(y2 - y1) * Fraction(x3 - x1))
    
    if val > 0:
        return "Counterclockwise"
    elif val < 0:
        return "Clockwise"
    else:
        return "Collinear"


def draw_implicit_line(a, b, c, painter, xlim=(-0.9, 0.4), **kwargs):
    x = np.linspace(*xlim, 500)
    if b != 0:
        y = (-a * x - c) / b
        painter.plot(x, y, **kwargs)
    else:
        x0 = -c / a
        painter.axvline(x0, **kwargs)


def plot_lines(t):
    
    # Min/max dimensions for visualisation
    xMin = 0
    xMax = 10
    yMin = 0
    yMax = 10
    
    xdMin = -2
    xdMax = 2
    ydMin = -2
    ydMax = 2
    
    
    # Input line segment
    pl = (1, 9)
    pi = (9, 1)

    # Point on the segment when we linearly interpolate with t \in [0, 1]
    i_li = (
        (1 - t) * pl[0] + t * pi[0],
        (1 - t) * pl[1] + t * pi[1],
    )
            
    # Compute dual point
    p_li = primal_line_to_dual_point(pl, pi)
    
        
    x = (p_li[0] - 50, p_li[0] + 50)
    i_li_a = (x[0], (-1.0 * i_li[0] * x[0] - 1) / i_li[1])
    i_li_b = (x[1], (-1.0 * i_li[0] * x[1] - 1) / i_li[1])

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))    

    # In the primal plane - draw the line segment and its endpoints
    ax1.plot([pl[0], pi[0]], [pl[1], pi[1]], marker='o', color='gray', markersize=10, linewidth=5.5)
    ax1.plot([pi[0]], [pi[1]], marker='o', color='green', markersize=10)
    ax1.plot([pl[0]], [pl[1]], marker='o', color='blue', markersize=10)

    # In the primal plane draw the lines that correspond to points along i_li* (in the dual plane)
    # In the dual plane draw the points the points along the line i_li*
    for t in np.linspace(0, 1, 500):
        # pt = (1 - t) * i_li_a[0] + t * i_li_a[0]
        newP = (
            (1 - t) * i_li_a[0] + t * p_li[0],
            (1 - t) * i_li_a[1] + t * p_li[1]
        )
        # print(newP)
        ax2.plot([newP[0]], [newP[1]], marker='o', color='teal', markersize=5)    
        draw_implicit_line(newP[0], newP[1], 1, ax1, xlim=(xMin, xMax), color='teal', linewidth=0.3)
    
    for t in np.linspace(0, 1, 500):
        # pt = (1 - t) * i_li_a[0] + t * i_li_a[0]
        newP = (
            # (1 - t) * p_li[0] + t * i_li_b[0],
            # (1 - t) * p_li[1] + t * i_li_b[1]
    
            (1 - t) * i_li_b[0] + t * p_li[0],
            (1 - t) * i_li_b[1] + t * p_li[1]
        )
        # print(newP)
        ax2.plot([newP[0]], [newP[1]], marker='o', color='purple', markersize=5)    
        draw_implicit_line(newP[0], newP[1], 1, ax1, xlim=(xMin, xMax), color='purple', linewidth=0.3)
    
    # ax1.set_title("Primal Plane (Segments)")
    ax1.set_xlabel("x")
    ax1.set_ylabel("y")
    ax1.set_aspect('equal')
    ax1.set_xlim(xMin, xMax)
    ax1.set_ylim(yMin, yMax)
    # ax1.set_xticklabels([])
    # ax1.set_yticklabels([])
    ax1.grid(True)

   
    # In the dual plane draw pi* an pl* and p_li*
    draw_implicit_line(pi[0], pi[1], 1, ax2, xlim = (-10, 10), color='green', linewidth=2.5)
    draw_implicit_line(pl[0], pl[1], 1, ax2, xlim = (-10, 10), color='blue', linewidth=2.5)
    ax2.plot([p_li[0]], [p_li[1]], marker='o', color='gray', markersize=12)
    
    # ax2.set_title("Dual Plane (Affine RP² Coordinates)")
    # ax2.set_xlabel("x (dual)")
    # ax2.set_ylabel("y (dual)")
    ax2.set_aspect('equal')
    ax2.grid(True)
    ax2.set_xlim(xdMin, xdMax)
    ax2.set_ylim(ydMin, ydMax)
    # ax2.set_xticklabels([])
    # ax2.set_yticklabels([])
    plt.tight_layout()
    plt.show()


interact(plot_lines, t=FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05));


interactive(children=(FloatSlider(value=0.5, description='t', max=1.0, step=0.05), Output()), _dom_classes=('w…

In [2]:
def draw_line_from_slope(m, b, painter, xlim=(-0.9, 0.4), **kwargs):
    """
    Draws a line y = m*x + b using the provided painter object.

    Parameters:
        m: slope of the line
        b: y-intercept of the line
        painter: matplotlib Axes object or similar with .plot method
        xlim: (min, max) tuple defining x-range
        kwargs: passed to painter.plot (e.g., color, linewidth, linestyle)
    """
    x = np.linspace(*xlim, 2)
    y = m * x + b
    painter.plot(x, y, **kwargs)


def primal_line_to_dual_point(p1, p2):
    """
    Given two points (x1, y1) and (x2, y2),
    returns the dual line in slope-intercept form: y = m x + b,
    where m = slope between p1 and p2, and b = -y-intercept.

    In the dual space, the line becomes a point: (m, -b)
    """
    x1, y1 = p1
    x2, y2 = p2

    if x1 == x2:
        raise ValueError("Vertical line — slope undefined in affine duality.")

    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1

    # In the dual space, the line y = mx + b becomes the point (m, -b)
    return (m, -b)



def plot_lines(t):
    xMin = 0
    xMax = 10
    yMin = 0
    yMax = 10

    # Dual plane bounds
    xdMin = -40
    xdMax = 40
    ydMin = -60
    ydMax = 20
    
    
    # Blue
    pl = (2, 2)
    pi = (8, 9)

    # Point along p_li
    pt = (
        (1 - t) * pl[0] + t * pi[0],
        (1 - t) * pl[1] + t * pi[1],
    )
    

    # Lines to points in the dual
    p_li = primal_line_to_dual_point(pl, pi)


    # get endpoints of the line i_bo* in x \in [-0.9, 0.4]
    x = (p_li[0] - (xdMax - xdMin), p_li[0] + (xdMax - xdMin))
    y = (pt[0] * x[0] - pt[1], pt[0] * x[1] - pt[1])

    i_bo_a = (x[0], y[0]) 
    i_bo_b = (x[1], y[1]) 
    
    
       
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))




    for t in np.linspace(0, 1, 1000):
        # pt = (1 - t) * i_bo_a[0] + t * i_bo_a[0]
        newP = (
            (1 - t) * i_bo_a[0] + t * p_li[0],
            (1 - t) * i_bo_a[1] + t * p_li[1]
        )
        # print(newP)
        ax2.plot([newP[0]], [newP[1]], marker='o', color='teal', markersize=5)    
        draw_line_from_slope(newP[0], -newP[1], ax1, xlim=(xMin, xMax), color='teal', linewidth=0.3)
    
    for t in np.linspace(0, 1, 1000):
        # pt = (1 - t) * i_bo_a[0] + t * i_bo_a[0]
        newP = (
            # (1 - t) * p_li[0] + t * i_bo_b[0],
            # (1 - t) * p_li[1] + t * i_bo_b[1]
    
            (1 - t) * i_bo_b[0] + t * p_li[0],
            (1 - t) * i_bo_b[1] + t * p_li[1]
        )
        # print(newP)
        ax2.plot([newP[0]], [newP[1]], marker='o', color='purple', markersize=5)    
        draw_line_from_slope(newP[0], -newP[1], ax1, xlim=(xMin, xMax), color='purple', linewidth=0.3)
    








    
    
    # Primal lines
    ax1.plot([pl[0], pi[0]], [pl[1], pi[1]], marker='o', color='blue', markersize=10)
    # ax1.plot([pv[0], pj[0]], [pv[1], pj[1]], marker='o', color='orange', markersize=10)
    # ax1.plot([pu[0], pk[0]], [pu[1], pk[1]], marker='o', color='green', markersize=10)

    ax1.plot([pt[0]], [pt[1]], marker='o', color='red', markersize=10)
    
    
    # ax1.set_title("Primal Plane (Segments)")
    ax1.set_xlabel("x")
    ax1.set_ylabel("y")
    ax1.set_aspect('equal')
    ax1.set_xlim(xMin, xMax)
    ax1.set_ylim(yMin, yMax)
    # ax1.set_xticklabels([])
    # ax1.set_yticklabels([])
    ax1.grid(True)

    #
    # Dual  Plane
    #
    
    # Draw the bounds of the edges
    draw_line_from_slope(pl[0], -pl[1], ax2, xlim = (xdMin, xdMax), color='blue', linewidth=0.5)
    draw_line_from_slope(pi[0], -pi[1], ax2, xlim = (xdMin, xdMax), color='blue', linewidth=0.5)
    
    # draw_line_from_slope(pt[0], -pt[1], ax2, xlim = (xdMin, xdMax), color='red', linewidth=0.5)
    
    # draw_implicit_line(i_bo[0], i_bo[1], 1, ax2, xlim = (-10, 10), color='black', linewidth=0.5)
    # draw_implicit_line(pt[0], pt[1], 1, ax2, xlim = (-10, 10), color='red', linewidth=0.5)
    # draw_implicit_line(pu[0], pu[1], 1, color='gree', linewidth=0.5)
    # draw_implicit_line(pk[0], pk[1], 1, color='green', linewidth=0.5)
    # Dual points
    
    
    # ax2.plot([p_li[0], p_vj[0]], [p_li[1], p_vj[1]], marker='o', color="black", linewidth=0.5)
    # ax2.plot([p_li[0], p_uk[0]], [p_li[1], p_uk[1]], marker='o', color="black", linewidth=0.5)
    # ax2.plot([p_li[0], p_u2k2[0]], [p_li[1], p_u2k2[1]], marker='o', color="black", linewidth=0.5)
    ax2.plot([p_li[0]], [p_li[1]], marker='o', color='blue', markersize=10)
    # ax2.plot([p_vj[0]], [p_vj[1]], marker='o', color='orange', markersize=10)
    # ax2.plot([p_uk[0]], [p_uk[1]], marker='o', color='green', markersize=10)

    ax2.plot([i_bo_a[0]], [i_bo_a[1]], marker='o', color='teal', markersize=10)
    ax2.plot([i_bo_b[0]], [i_bo_b[1]], marker='o', color='purple', markersize=10)
    
    # ax2.plot([i_bo_a[0]], [i_bo_a[1]], marker='o', color='teal', markersize=10)
    # ax2.plot([i_bo_b[0]], [i_bo_b[1]], marker='o', color='purple', markersize=10)


    
    # ax2.set_title("Dual Plane (Affine RP² Coordinates)")
    # ax2.set_xlabel("x (dual)")
    # ax2.set_ylabel("y (dual)")
    ax2.set_aspect('equal')
    ax2.grid(True)
    ax2.set_xlim(xdMin, xdMax)
    ax2.set_ylim(ydMin, ydMax)
    # ax2.set_xticklabels([])
    # ax2.set_yticklabels([])
    plt.tight_layout()
    plt.show()


# plot_lines(0.2)
interact(plot_lines, t=FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05));

interactive(children=(FloatSlider(value=0.5, description='t', max=1.0, step=0.05), Output()), _dom_classes=('w…